## Importing Relevant Libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Loading the data

In [13]:
mnist_dataset, mnist_info = tfds.load('mnist', with_info = True, as_supervised = True)
## mnist_info contains all the information about the mnist dataset such as number of training and testing samples
## as_supervised = True loads the data in 2 tuple format (one containing inputs, second containing targets)

mnist_train , mnist_test = mnist_dataset['train'] , mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
## num_validation_samples contains the number of validation samples, not the actual data but the number only

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


## Preprocessing the data

## Scaling the data

In [29]:
def scale (image,label):
    image = tf.cast(image, tf.float32)
    image = image/255.
    return image,label

## this function helps us scale the input data (training and test data cantaining numbers between 0 and 255 representing different shades of grey to form a number) into a number between 0 and 1

scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_train.map(scale)

## .map(*function name*) here is used to transform input data in any way we like, it takes in argument a function

## Shuffling the data

In [30]:
Buffer_size = 10000
## number of observations selected at a time to be shuffled

shuffled_and_scaled_train_validation_data = scaled_train_validation_data.shuffle(Buffer_size)

## Assigning the data to variables

In [31]:
validation_data = shuffled_and_scaled_train_validation_data.take(num_validation_samples)
## .take() takes out a particular number of samples from the dataset

training_data = shuffled_and_scaled_train_validation_data.skip(num_validation_samples)
## .skip() leaves behind a particular number of samples from dataset and keeps the rest 

## Setting Batch size

In [32]:
batch_size = 100
## batch size decides the number of batches in which our observations will be divided into

training_data = training_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_input, validation_target = next(iter(validation_data))

## Outlining the model

In [62]:
output_size = 10
input_size = 784
hidden_layer_size = 50

model = tf.keras.Sequential([
                    tf.keras.layers.Flatten(input_shape = (28,28,1)), ##converts tensor to vector so that our model can work on it
                    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'), ## first hidden layer with Relu activation function
                    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'), ## second hidden layer with Relu activation function
                    tf.keras.layers.Dense(output_size,activation = 'softmax') ## output layer with softmax activation function because we want the output as a probability
                    ])

## Choosing the optimizer and loss function

In [63]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
## sparse_categorical_crossentropy loss function 

## Training the model

In [64]:
num_epochs = 5
model.fit(training_data, epochs = num_epochs,validation_data=(validation_input, validation_target), verbose =2)

Epoch 1/5
540/540 - 10s - loss: 0.4247 - accuracy: 0.8819 - val_loss: 0.2332 - val_accuracy: 0.9370
Epoch 2/5
540/540 - 5s - loss: 0.1939 - accuracy: 0.9440 - val_loss: 0.1710 - val_accuracy: 0.9497
Epoch 3/5
540/540 - 5s - loss: 0.1504 - accuracy: 0.9565 - val_loss: 0.1457 - val_accuracy: 0.9577
Epoch 4/5
540/540 - 5s - loss: 0.1226 - accuracy: 0.9633 - val_loss: 0.1239 - val_accuracy: 0.9643
Epoch 5/5
540/540 - 6s - loss: 0.1040 - accuracy: 0.9678 - val_loss: 0.1058 - val_accuracy: 0.9692


## Testing our model

In [40]:
test_loss , test_accuracy = model.evaluate(test_data)

6/6 [==============================] - 6s 889ms/step - loss: 0.0401 - accuracy: 0.9887


In [42]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.04. Test accuracy: 98.87%


## Conclusion

Our model has an accuracy of 98.78% in the real world